In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Ensure TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set up mixed precision training for speed
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
df_ = pd.read_csv("all_buildings_data.csv")

In [ ]:
# Afficher les premières lignes du DataFrame
print(df_.head())

# Afficher les informations générales (colonnes, types de données, valeurs manquantes)
print(df_.info())

# Résumé statistique
print(df_.describe())


In [ ]:
df = df_.drop_duplicates()

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'building'])

In [ ]:
# Extract features and target
X = df.drop(columns=['aggregate']).values
y = df['aggregate'].values

In [ ]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Reduce dimensions using PCA (optional but can speed up training)
pca = PCA(n_components=20)  # Reduce to 20 components
X = pca.fit_transform(X)

In [ ]:
# Reshape X for Conv1D input
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the optimized Seq2Point model
def seq2point_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv1D(16, 3, activation='relu', padding='same'))
    model.add(Conv1D(32, 3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mse', optimizer=RMSprop(learning_rate=0.001), metrics=['mae'])
    return model

In [ ]:
# Create the model
model = seq2point_model((X_train.shape[1], 1))

In [ ]:
# Set up EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Train the model with larger batch size (if GPU is available)
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.1,
          verbose=1, callbacks=[early_stopping])

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Model Mean Absolute Error: {mae}')

In [ ]:
# Function to predict appliance consumption from aggregate features
def predict_appliance_consumption(aggregate_features, scaler, model):
    # Standardize the input features
    aggregate_features = np.array(aggregate_features).reshape(1, -1)
    aggregate_features = scaler.transform(aggregate_features)
    aggregate_features = aggregate_features.reshape((aggregate_features.shape[0], aggregate_features.shape[1], 1))

    # Predict using the trained model
    predictions = model.predict(aggregate_features)
    return predictions